In [4]:


import sqlite3
import pandas as pd
import numpy as np
import string
import pandas as pd
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split,cross_val_score
from sklearn.metrics import accuracy_score,log_loss,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
import matplotlib.pyplot as plt
 




In [5]:
df=pd.read_csv(".\\reviews.csv")

In [6]:
#print(df.shape)
#print(df.columns)
df.dropna(inplace=True)
df=df[df.Score != 3]
print(df['Score'].value_counts())
positive_or_negative=lambda a: "positive" if ((type(a)== type(2)) and (a > 3)) else "negative"
df['Score']=df['Score'].map(positive_or_negative)
print(df['Score'].value_counts())

5    363111
4     80655
1     52264
2     29743
Name: Score, dtype: int64
positive    443766
negative     82007
Name: Score, dtype: int64


In [7]:
##soring data according product id
df_sorted=df.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

##deduplication of entries
final=df_sorted.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]


print(final.shape)

#check how much data is present, len(final['ProductId'])
#print(final['ProductId'].size[0]*1.0/df['ProductId'].size[0]*1.0)*100

(364159, 10)


In [8]:
final.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
150522,150523,0006641040,A2P4F2UO0UMP8C,"Elizabeth A. Curry ""Lovely Librarian""",0,0,positive,1096675200,MMMM chicken soup....,Summary: A young boy describes the usefulness...


In [9]:
final['Score'].value_counts()
###taking sample
final_sample= final.sample(frac=0.01)
df_sorted=final_sample.sort_values('Time')
print(df_sorted.shape)




(3642, 10)


In [10]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# loading stop words from nltk library
stop_words = set(stopwords.words('english'))
sno = SnowballStemmer('english')

In [11]:
df_sorted['newText']=np.nan
df_sorted.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,newText
102193,102194,B0000UD67Y,A18O1KPT80HUDQ,"K. Moore ""collegian""",0,0,positive,1072656000,Simply Chai,I've tried many of packaged Chai products - li...,NaN


In [12]:
def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        final_string=""
        # replace every special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # replace multiple spaces with single space
        total_text = re.sub('\s+',' ', total_text)
        # converting all the chars into lower-case.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                #word=sno(word)
                string += word + " "
        
        df_sorted.loc[index,column] = string
        #print(string)
        #final_string.append(string)

In [13]:
df_sorted['NewText']=None
##slicing for simplicity
#df_sorted=df_sorted.iloc[0:100,:]
for index, row in df_sorted.iterrows():
    nlp_preprocessing(df_sorted['Text'][index], index, 'NewText')



In [14]:
df_sorted.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,newText,NewText
102193,102194,B0000UD67Y,A18O1KPT80HUDQ,"K. Moore ""collegian""",0,0,positive,1072656000,Simply Chai,I've tried many of packaged Chai products - li...,NaN,tried many packaged chai products liquid dry s...
375993,375994,B0000DBN1H,A281NPSIMI1C2R,"Rebecca of Amazon ""The Rebecca Review""",2,4,positive,1073433600,English Breakfast Style - High Quality Tea,"""There are few hours in life more agreeable th...",NaN,hours life agreeable hour dedicated ceremony k...


In [15]:
def sent_vect(list_of_sent):
    sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
    for sent in list_of_sent: # for each review/sentence
        sent_vec = np.zeros(50) # as word vectors are of zero length
        cnt_words =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            try:
                vec = w2v_model.wv[word]
                sent_vec += vec
                cnt_words += 1
            except:
                pass
        sent_vec /= cnt_words
        sent_vectors.append(sent_vec)
    print(len(sent_vectors))
    print(len(sent_vectors[0]))
    return sent_vectors

In [16]:
##split sentenses to list
list_of_sent=[]
def list_sentense(df_sorted):
    for sent in df_sorted['NewText'].values:
        sentence=sent.split()
        list_of_sent.append(sentence)
    return list_of_sent

list_sentenses=list_sentense(df_sorted)
print(list_sentenses[2])
print(len(list_sentenses))
print(df_sorted['NewText'].shape)


['weekend', 'new', 'senseo', 'coffee', 'maker', 'report', 'delight', 'certainly', 'consider', 'connoisseur', 'coffee', 'spending', 'average', '30', 'week', 'starbucks', 'makes', 'least', 'average', 'coffee', 'consumer', 'br', 'br', 'agree', 'reviewers', 'taste', 'comparable', 'freshly', 'made', 'starbucks', 'senseo', 'home', 'coffee', 'maker', '70', 'starbucks', 'controls', 'every', 'element', 'process', 'uses', 'significantly', 'sophisticated', 'equipment', 'importantly', 'senseo', 'worth', 'money', 'br', 'br', 'work', 'home', 'businessperson', 'enjoy', 'fresh', 'cup', 'desk', 'frothy', 'result', 'senseo', 'simulates', 'latte', 'add', 'hot', 'milk', 'final', 'cup', 'coffee', 'never', 'put', 'milk', 'senseo', 'br', 'br', 'dark', 'roast', 'favorite', 'find', 'switch', 'medium', 'roast', 'decaf', 'first', 'cup', 'keep', 'bouncing', 'walls', 'br', 'br', 'said', 'equally', 'substitue', 'fresh', 'starbucks', 'fast', 'enjoyable', 'easy', 'use', 'cost', 'pods', 'issue', 'makes', 'perfect', 's

In [17]:
##creating our own word2vec model
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
w2v_model=gensim.models.Word2Vec(list_sentenses,min_count=5,size=50, workers=4)

C:\Users\admin\AppData\Local\Programs\Python\Python36\Lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
print(len(list_sentenses))
df_sorted.shape
df_sorted['list_Sentenses']=list_sentenses
df_sorted.head(1)

3642


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,newText,NewText,list_Sentenses
102193,102194,B0000UD67Y,A18O1KPT80HUDQ,"K. Moore ""collegian""",0,0,positive,1072656000,Simply Chai,I've tried many of packaged Chai products - li...,NaN,tried many packaged chai products liquid dry s...,"[tried, many, packaged, chai, products, liquid..."


In [19]:

size=int(df_sorted.shape[0]*0.7)
df_train=df_sorted.iloc[:size,:]
df_test=df_sorted.iloc[size:,]
#df_train= df_train.sample(frac=0.01)
#df_test= df_test.sample(frac=0.01)


In [20]:
#print(df_train.head(1))
#print(df_test.head(1))
df_train.shape
df_train.columns.values
#df_train.NewText

array(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text',
       'newText', 'NewText', 'list_Sentenses'], dtype=object)

In [21]:
y_train=df_train['Score']
y_test=df_test['Score']
df_train['Score'].value_counts()

positive    2159
negative     390
Name: Score, dtype: int64

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer(ngram_range=(1,3))
train_tf_idf = tf_idf_vect.fit_transform(df_train['NewText'].values)
test_tf_idf=tf_idf_vect.transform(df_test['NewText'].values)
tfidf_feat = tf_idf_vect.get_feature_names()
print(len(tfidf_feat))
tfidf_feat[100000:100010]

181494


['markets site',
 'markets site would',
 'markets texas',
 'markets texas 99',
 'markets usually',
 'markets usually find',
 'marks',
 'marks weak',
 'marks weak watery',
 'marmalade']

In [50]:
import pdb
# TF-IDF weighted Word2Vec
def tfidf_word2vec(w2v_model,list_of_sent,final_tf_idf):
    tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
    row=0;
    for sent in list_of_sent: # for each review/sentence
        sent_vec = np.zeros(50) # as word vectors are of zero length
        weight_sum =0; # num of words with a valid vector in the sentence/review
        for word in sent: # for each word in a review/sentence
            try:
                vec = w2v_model.wv[word]
            
                # obtain the tf_idfidf of a word in a sentence/review
                
                tfidf = final_tf_idf[row, tfidf_feat.index(word)]
                sent_vec += (vec * tfidf)
                weight_sum += tfidf
        #pdb.set_trace() 
        #print(weight_sum)
            except:
                pass
        if weight_sum != 0:
            sent_vec /= weight_sum
        #pdb.set_trace() 
        tfidf_sent_vectors.append(sent_vec)
        row += 1
        #print("1 sentense completed",tfidf_sent_vectors)
        print("row..",row)
        
    return tfidf_sent_vectors


In [51]:

tfidfword2vec_train_vectors=tfidf_word2vec(w2v_model,df_train['list_Sentenses'],train_tf_idf)


row.. 1
row.. 2
row.. 3
row.. 4
row.. 5
row.. 6
row.. 7
row.. 8
row.. 9
row.. 10
row.. 11
row.. 12
row.. 13
row.. 14
row.. 15
row.. 16
row.. 17
row.. 18
row.. 19
row.. 20
row.. 21
row.. 22
row.. 23
row.. 24
row.. 25
row.. 26
row.. 27
row.. 28
row.. 29
row.. 30
row.. 31
row.. 32
row.. 33
row.. 34
row.. 35
row.. 36
row.. 37
row.. 38
row.. 39
row.. 40
row.. 41
row.. 42
row.. 43
row.. 44
row.. 45
row.. 46
row.. 47
row.. 48
row.. 49
row.. 50
row.. 51
row.. 52
row.. 53
row.. 54
row.. 55
row.. 56
row.. 57
row.. 58
row.. 59
row.. 60
row.. 61
row.. 62
row.. 63
row.. 64
row.. 65
row.. 66
row.. 67
row.. 68
row.. 69
row.. 70
row.. 71
row.. 72
row.. 73
row.. 74
row.. 75
row.. 76
row.. 77
row.. 78
row.. 79
row.. 80
row.. 81
row.. 82
row.. 83
row.. 84
row.. 85
row.. 86
row.. 87
row.. 88
row.. 89
row.. 90
row.. 91
row.. 92
row.. 93
row.. 94
row.. 95
row.. 96
row.. 97
row.. 98
row.. 99
row.. 100
row.. 101
row.. 102
row.. 103
row.. 104
row.. 105
row.. 106
row.. 107
row.. 108
row.. 109
row.. 110
row.. 11

row.. 833
row.. 834
row.. 835
row.. 836
row.. 837
row.. 838
row.. 839
row.. 840
row.. 841
row.. 842
row.. 843
row.. 844
row.. 845
row.. 846
row.. 847
row.. 848
row.. 849
row.. 850
row.. 851
row.. 852
row.. 853
row.. 854
row.. 855
row.. 856
row.. 857
row.. 858
row.. 859
row.. 860
row.. 861
row.. 862
row.. 863
row.. 864
row.. 865
row.. 866
row.. 867
row.. 868
row.. 869
row.. 870
row.. 871
row.. 872
row.. 873
row.. 874
row.. 875
row.. 876
row.. 877
row.. 878
row.. 879
row.. 880
row.. 881
row.. 882
row.. 883
row.. 884
row.. 885
row.. 886
row.. 887
row.. 888
row.. 889
row.. 890
row.. 891
row.. 892
row.. 893
row.. 894
row.. 895
row.. 896
row.. 897
row.. 898
row.. 899
row.. 900
row.. 901
row.. 902
row.. 903
row.. 904
row.. 905
row.. 906
row.. 907
row.. 908
row.. 909
row.. 910
row.. 911
row.. 912
row.. 913
row.. 914
row.. 915
row.. 916
row.. 917
row.. 918
row.. 919
row.. 920
row.. 921
row.. 922
row.. 923
row.. 924
row.. 925
row.. 926
row.. 927
row.. 928
row.. 929
row.. 930
row.. 931
row.. 932


row.. 1593
row.. 1594
row.. 1595
row.. 1596
row.. 1597
row.. 1598
row.. 1599
row.. 1600
row.. 1601
row.. 1602
row.. 1603
row.. 1604
row.. 1605
row.. 1606
row.. 1607
row.. 1608
row.. 1609
row.. 1610
row.. 1611
row.. 1612
row.. 1613
row.. 1614
row.. 1615
row.. 1616
row.. 1617
row.. 1618
row.. 1619
row.. 1620
row.. 1621
row.. 1622
row.. 1623
row.. 1624
row.. 1625
row.. 1626
row.. 1627
row.. 1628
row.. 1629
row.. 1630
row.. 1631
row.. 1632
row.. 1633
row.. 1634
row.. 1635
row.. 1636
row.. 1637
row.. 1638
row.. 1639
row.. 1640
row.. 1641
row.. 1642
row.. 1643
row.. 1644
row.. 1645
row.. 1646
row.. 1647
row.. 1648
row.. 1649
row.. 1650
row.. 1651
row.. 1652
row.. 1653
row.. 1654
row.. 1655
row.. 1656
row.. 1657
row.. 1658
row.. 1659
row.. 1660
row.. 1661
row.. 1662
row.. 1663
row.. 1664
row.. 1665
row.. 1666
row.. 1667
row.. 1668
row.. 1669
row.. 1670
row.. 1671
row.. 1672
row.. 1673
row.. 1674
row.. 1675
row.. 1676
row.. 1677
row.. 1678
row.. 1679
row.. 1680
row.. 1681
row.. 1682
row.. 1683

row.. 2339
row.. 2340
row.. 2341
row.. 2342
row.. 2343
row.. 2344
row.. 2345
row.. 2346
row.. 2347
row.. 2348
row.. 2349
row.. 2350
row.. 2351
row.. 2352
row.. 2353
row.. 2354
row.. 2355
row.. 2356
row.. 2357
row.. 2358
row.. 2359
row.. 2360
row.. 2361
row.. 2362
row.. 2363
row.. 2364
row.. 2365
row.. 2366
row.. 2367
row.. 2368
row.. 2369
row.. 2370
row.. 2371
row.. 2372
row.. 2373
row.. 2374
row.. 2375
row.. 2376
row.. 2377
row.. 2378
row.. 2379
row.. 2380
row.. 2381
row.. 2382
row.. 2383
row.. 2384
row.. 2385
row.. 2386
row.. 2387
row.. 2388
row.. 2389
row.. 2390
row.. 2391
row.. 2392
row.. 2393
row.. 2394
row.. 2395
row.. 2396
row.. 2397
row.. 2398
row.. 2399
row.. 2400
row.. 2401
row.. 2402
row.. 2403
row.. 2404
row.. 2405
row.. 2406
row.. 2407
row.. 2408
row.. 2409
row.. 2410
row.. 2411
row.. 2412
row.. 2413
row.. 2414
row.. 2415
row.. 2416
row.. 2417
row.. 2418
row.. 2419
row.. 2420
row.. 2421
row.. 2422
row.. 2423
row.. 2424
row.. 2425
row.. 2426
row.. 2427
row.. 2428
row.. 2429

In [54]:
tfidfword2vec_train_vectors[2]

array([ 0.40750156,  0.22457858, -0.44698564,  0.44767593, -0.11897147,
       -0.18169751,  0.75846915,  0.20094814,  0.14236499, -0.17973203,
        0.60115393, -0.03611602, -0.40477143, -0.03674163,  0.04619242,
       -0.45179496,  0.60982637, -0.53409608,  0.17815753,  0.30472455,
        0.19887166, -0.1772762 ,  0.004608  , -0.22437061, -0.25047824,
       -0.21857883, -0.27265858, -0.37913847, -0.37940054,  0.42393587,
       -0.65220939, -0.32734618,  0.6052916 , -0.03742982, -0.28900674,
       -0.12438759, -0.35057444,  0.26313782,  0.20413692, -0.79099498,
        0.15249869,  0.42378793,  0.22196594,  0.13596012,  0.15696272,
        0.31970607,  0.33169853,  0.18205057,  0.10671623,  0.09904266])

In [55]:
tfidfword2vec_test_vectors=tfidf_word2vec(w2v_model,df_test['list_Sentenses'],test_tf_idf)

row.. 1
row.. 2
row.. 3
row.. 4
row.. 5
row.. 6
row.. 7
row.. 8
row.. 9
row.. 10
row.. 11
row.. 12
row.. 13
row.. 14
row.. 15
row.. 16
row.. 17
row.. 18
row.. 19
row.. 20
row.. 21
row.. 22
row.. 23
row.. 24
row.. 25
row.. 26
row.. 27
row.. 28
row.. 29
row.. 30
row.. 31
row.. 32
row.. 33
row.. 34
row.. 35
row.. 36
row.. 37
row.. 38
row.. 39
row.. 40
row.. 41
row.. 42
row.. 43
row.. 44
row.. 45
row.. 46
row.. 47
row.. 48
row.. 49
row.. 50
row.. 51
row.. 52
row.. 53
row.. 54
row.. 55
row.. 56
row.. 57
row.. 58
row.. 59
row.. 60
row.. 61
row.. 62
row.. 63
row.. 64
row.. 65
row.. 66
row.. 67
row.. 68
row.. 69
row.. 70
row.. 71
row.. 72
row.. 73
row.. 74
row.. 75
row.. 76
row.. 77
row.. 78
row.. 79
row.. 80
row.. 81
row.. 82
row.. 83
row.. 84
row.. 85
row.. 86
row.. 87
row.. 88
row.. 89
row.. 90
row.. 91
row.. 92
row.. 93
row.. 94
row.. 95
row.. 96
row.. 97
row.. 98
row.. 99
row.. 100
row.. 101
row.. 102
row.. 103
row.. 104
row.. 105
row.. 106
row.. 107
row.. 108
row.. 109
row.. 110
row.. 11

row.. 834
row.. 835
row.. 836
row.. 837
row.. 838
row.. 839
row.. 840
row.. 841
row.. 842
row.. 843
row.. 844
row.. 845
row.. 846
row.. 847
row.. 848
row.. 849
row.. 850
row.. 851
row.. 852
row.. 853
row.. 854
row.. 855
row.. 856
row.. 857
row.. 858
row.. 859
row.. 860
row.. 861
row.. 862
row.. 863
row.. 864
row.. 865
row.. 866
row.. 867
row.. 868
row.. 869
row.. 870
row.. 871
row.. 872
row.. 873
row.. 874
row.. 875
row.. 876
row.. 877
row.. 878
row.. 879
row.. 880
row.. 881
row.. 882
row.. 883
row.. 884
row.. 885
row.. 886
row.. 887
row.. 888
row.. 889
row.. 890
row.. 891
row.. 892
row.. 893
row.. 894
row.. 895
row.. 896
row.. 897
row.. 898
row.. 899
row.. 900
row.. 901
row.. 902
row.. 903
row.. 904
row.. 905
row.. 906
row.. 907
row.. 908
row.. 909
row.. 910
row.. 911
row.. 912
row.. 913
row.. 914
row.. 915
row.. 916
row.. 917
row.. 918
row.. 919
row.. 920
row.. 921
row.. 922
row.. 923
row.. 924
row.. 925
row.. 926
row.. 927
row.. 928
row.. 929
row.. 930
row.. 931
row.. 932
row.. 933


In [57]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,precision_recall_curve
##find optimal k for kdtree, using gridSerachCV
from sklearn.model_selection import GridSearchCV
knn_model=KNeighborsClassifier()
neighbours=list(range(1,50,2))
param_grid = {'n_neighbors':neighbours}
grid = GridSearchCV(knn_model, param_grid,cv=10,scoring = 'accuracy')

grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)

{'n_neighbors': 25}


In [58]:
optimal_k=grid.best_params_['n_neighbors']
optimal_k

25

In [62]:
knn=KNeighborsClassifier(n_neighbors=optimal_k,algorithm='kd_tree')
knn.fit(tfidfword2vec_train_vectors,y_train)
pred=knn.predict(tfidfword2vec_test_vectors)
accuracy=accuracy_score(y_test,pred)
print("the accuracy of KNN on word2vec data is %f"%(accuracy) )

the accuracy of KNN on word2vec data is 0.826167


In [61]:
##using linear svc
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC,LinearSVC
C = [10 ** x for x in range(-5, 1)]
param_grid = {'C':C}
grid = GridSearchCV(LinearSVC(class_weight='balanced'), param_grid,cv=10,scoring = 'accuracy')
grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)
best_C=grid.best_params_['C']

{'C': 1e-05}


In [66]:
svc= LinearSVC(C=best_C,random_state=42,class_weight='balanced')
svc.fit(tfidfword2vec_train_vectors,y_train)
pred=svc.predict(tfidfword2vec_test_vectors)
print(pred)
accuracy_score(y_test,pred)
print("the accuracy of linear svc on word2vec data is %f"%(accuracy) )

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']
the accuracy of linear svc on word2vec data is 0.826167


In [67]:
##using RBFSVC


C = [10 ** x for x in range(-5, 1)]
param_grid = {'C':C}
grid = GridSearchCV(SVC(class_weight='balanced'), param_grid,cv=10,scoring = 'accuracy')
grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)
best_C=grid.best_params_['C']

{'C': 1e-05}


In [68]:
svc= SVC(C=best_C,random_state=42,class_weight='balanced')
svc.fit(tfidfword2vec_train_vectors,y_train)
pred=svc.predict(tfidfword2vec_test_vectors)
print(pred)
accuracy_score(y_test,pred)
print("the accuracy of linear svc on word2vec data is %f"%(accuracy) )

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']
the accuracy of linear svc on word2vec data is 0.826167


In [ ]:
##Logistic regression

In [70]:
####logistic regression
from sklearn.linear_model import LogisticRegression
C = [10 ** x for x in range(-6, 3)]
param_grid = {'C':C}
grid = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid,cv=10,scoring = 'accuracy')
grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)
best_C=grid.best_params_['C']

{'C': 1e-06}


In [71]:
svc= LogisticRegression(C=best_C,random_state=42,class_weight='balanced')
svc.fit(tfidfword2vec_train_vectors,y_train)
pred=svc.predict(tfidfword2vec_test_vectors)
print(pred)
accuracy_score(y_test,pred)
print("the accuracy of logistic regression on word2vec data is %f"%(accuracy) )

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']
the accuracy of logistic regression on word2vec data is 0.826167


In [ ]:
###random Forest

In [79]:
from sklearn.ensemble import RandomForestClassifier
C = [10 ** x for x in range(-6, 3)]

n_estimators_range = [10,100,200,500,1000,2000]
max_depth_range = [5, 20]
param_grid = {'n_estimators':n_estimators_range,'max_depth':max_depth_range}
grid = GridSearchCV(RandomForestClassifier(class_weight='balanced'), param_grid,cv=10,scoring = 'accuracy')


In [80]:
grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)
optimal_n_estimators=grid.best_params_['n_estimators']
optimal_max_depth=grid.best_params_['max_depth']

{'max_depth': 20, 'n_estimators': 500}


In [82]:
rfc= RandomForestClassifier(class_weight='balanced',n_estimators=optimal_n_estimators,max_depth=optimal_max_depth)
rfc.fit(tfidfword2vec_train_vectors,y_train)
rfc_pred=rfc.predict(tfidfword2vec_test_vectors)
print(pred)
accuracy_score(y_test,rfc_pred)
print("the accuracy of logistic regression on word2vec data is %f"%(accuracy) )

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']
the accuracy of logistic regression on word2vec data is 0.826167


In [89]:
confusion_matrix(df_test['Score'].values,rfc_pred)

array([[  2, 188],
       [  1, 902]], dtype=int64)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
'''
C = [10 ** x for x in range(-6, 3)]

n_estimators_range = [10,100,200,500,1000,2000]
max_depth_range = [5, 10]
param_grid = {'n_estimators':n_estimators_range,'max_depth':max_depth_range}
grid = GridSearchCV(GradientBoostingClassifier(class_weight='balanced'), param_grid,cv=10,scoring = 'accuracy')
grid.fit(tfidfword2vec_train_vectors,y_train)
print(grid.best_params_)
optimal_n_estimators=grid.best_params_['n_estimators']
optimal_max_depth=grid.best_params_['max_depth']
'''

In [87]:
gbc= GradientBoostingClassifier(n_estimators=500,max_depth=5)
gbc.fit(tfidfword2vec_train_vectors,df_train['Score'].values)
gbc_pred=gbc.predict(tfidfword2vec_test_vectors)
print(gbc_pred)
accuracy_score(df_test['Score'].values,gbc_pred)
print("the accuracy of GBDT on word2vec data is %f"%(accuracy) )

['positive' 'positive' 'positive' ... 'positive' 'positive' 'positive']
the accuracy of GBDT on word2vec data is 0.826167


In [88]:
confusion_matrix(df_test['Score'].values,gbc_pred)

array([[  9, 181],
       [ 15, 888]], dtype=int64)